In [1]:
!pip install pandas sqlalchemy sqlacodegen

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pickle
import pandas as pd

In [3]:
# The pickle module implements binary protocols 
# for serializing and de-serializing a Python object structure
data  = [{"name":"elfi"}]
# 'b' appended to the mode opens the file in binary mode,
# so there are also modes like 'rb', 'wb', and 'r+b'. 
with open("users.pk1", "wb")as f:
    pickle.dump(data, f)

In [4]:
data_from_pk1 = pickle.load(open("users.pk1","rb"))
print(data_from_pk1, type(data_from_pk1))

[{'name': 'elfi'}] <class 'list'>


In [5]:
df = pd.DataFrame(data)
df.head()

,name
0,elfi


In [6]:
df.to_pickle("df_pk1")
df.dtypes

name    object
dtype: object

In [7]:
df2 = pd.read_pickle("df_pk1")
df2.head()

,name
0,elfi


In [8]:
df2.dtypes

name    object
dtype: object

### SQLAlchemy

In [9]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str = 'Unknown'
    genre:str = 'Action'
    year:int  = None
    
#     def __init__(self, name='',genre="",  *args, **kwargs):
#         super().__init__( *args, **kwargs)
#         self.name = name

In [10]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
movie_obj.name

'Interstellar'

In [11]:
data = [{
    "name": "Interstellar",
    "genre": "Sci-Fi"
},
{
    "name": "The Martian",
    "genre": "Sci-Fi"
},
{
    "name": "Arrival",
    "genre": "Sci-Fi"
}
]

In [12]:
df = pd.DataFrame(data)
df.head()

,name,genre
0,Interstellar,Sci-Fi
1,The Martian,Sci-Fi
2,Arrival,Sci-Fi


In [13]:
import sqlalchemy
from sqlalchemy import Column, Integer, String
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [14]:
engine = create_engine("sqlite:///app.db") # mysql, postgres

In [15]:
Session = sessionmaker(bind=engine)
my_sess = Session()

In [16]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [17]:
class Movie(Base): # table
    __tablename__ = "movies"
    
    id = Column(Integer, primary_key=True) # auto created for us
    name = Column(String) # 'Unknown' # col
    genre = Column(String) # 'Action' # col
    description = Column(String) # 'Action' # col
    year = Column(Integer, nullable=True) # None # col
    
    
    def __repr__(self):
        return f"<Movie name={self.name}>"
        
# saved instance = row

In [18]:
# Add table(s) to database
Base.metadata.create_all(engine)

In [24]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
print(movie_obj.name)
my_sess.add(movie_obj) # prepare to save
my_sess.commit() #save

Interstellar


In [23]:
movie_obj.id

3

In [25]:
movie_obj2 = Movie(name='The Martian', genre='Sci-Fi')
movie_obj3 = Movie(name='Inception', genre='Sci-Fi')
my_sess.add(movie_obj3)
my_sess.add(movie_obj2)
my_sess.commit()

In [26]:
print(movie_obj2.id, movie_obj3.id)

6 5


### CRUD in SQLAlchemy¶

In [27]:
Session = sessionmaker(bind=engine)
session = Session()

In [28]:
# Create
movie = Movie(name = "Alian")
session.add(movie)
session.commit()

In [35]:
# Retreive
# Get 1 item
movie_A = session.query(Movie).get(2)
# Get all 
movies = session.query(Movie).all()
# List and Filter
movies = session.query(Movie).filter_by(name = "Alian").all()
# Filter by column value
movies = session.query(Movie).filter(Movie.name.contains("Alian")).all()
movies

[<Movie name=Alian>]

In [37]:
# Update
movie_A = session.query(Movie).get(2)
movie_A.name = "Updated name"
session.commit()
# 
movies = session.query(Movie).filter(Movie.name.contains("Alian")).all()
for movie in movies:
    movie.name = "Alien"
session.commit()

In [38]:
# Delete
movie_A = session.query(Movie).get(2)
session.delete(movie_A)
session.commit()

In [39]:
session.flush()#clear all

### SQLAlchemy & Pandas

In [53]:
import sqlacodegen
# This is a tool that reads the structure of an existing database 
# and generates the appropriate SQLAlchemy model code, 
# using the declarative style if possible.

In [61]:
!sqlacodegen sqlite:///app.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)


In [62]:
engine = create_engine("sqlite:///app.db")
Session = sessionmaker(bind=engine)
session = Session()

In [63]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp2(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f"<Movie name={self.name}>"

In [64]:
qs = session.query(MovieApp2).all()

In [66]:
for old_obj in qs:
    movie_obj = Movie(name=old_obj.name, genre=old_obj.genre, year=old_obj.year)
    print(movie_obj.name)
    my_sess.add(movie_obj)
my_sess.commit()

Interstellar
Interstellar
Interstellar
Inception
The Martian
Alien


In [68]:
old_engine = create_engine("sqlite:///app.db")
old_df = pd.read_sql_table("movies", old_engine)
old_df.head()

,id,name,genre,description,year
0,1,Interstellar,Sci-Fi,None,None
1,3,Interstellar,Sci-Fi,None,None
2,4,Interstellar,Sci-Fi,None,None
3,5,Inception,Sci-Fi,None,None
4,6,The Martian,Sci-Fi,None,None
